# 11. 인터페이스 : 프로토콜에서 ABC까지

## Intro. 인터페이스 ??


__In wiki__

 인터페이스는 서로 다른 두 개 이상의 독립된 컴퓨터 시스템 구성 요소 간에 정보를 교환하는 공유 경계입니다. 컴퓨터와 사용자 간의 통신이 가능하도록 하는 장치나 프로그램을 의미하기도 합니다. 
 
__In OOP__

인터페이스는 여러 프로그래밍 언어들이 추상화라는 OOP개념을 더 적절하게 사용하기 위해 문법적으로 지원해주는 논리적 도구입니다.

__Joel__

사용법

## 11. 1 파이썬 문화에서의 인터페이스와 프로토콜

__In Fluent Python__

클래스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)들의 집합이 인터페이스입니다. 여기에는 특별 메서드도 포함됩니다.

파이썬에서는 보호된 속성(`_attr`) 이나 비공개 속성도 쉽게 접근할 수 있지만, 이는 __인터페이스에 속하지 않습니다.__ 일종의 관례지만 지켜줍시다 !

공개 데이터 속성을 객체의 인터페이스로 사용하면 언제나 데이터 속성을 호출 코드를 망가뜨리지 않고 `obj.attr` 구문을 이용해서 getter/setter를 구현하는 프로퍼티(멤버)로 변환할 수 있습니다.

In [1]:
## ex 11-1
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [2]:
my_vector = Vector2d(1,2)
my_vector.x

1.0

In [3]:
## ex 11-2
class Vector2d_2:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [4]:
my_vector2 = Vector2d_2(1, 2)
my_vector2.x

1.0

아래의 코드는 위의 코드의 프로퍼티를 read-only로 바꿨지만, 인터페이스의 핵심은 바꾸지 않았습니다. 

인터페이스 정의에 __시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부__ 라고 설명을 보충할 수 있습니다.

어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스를 스몰토크에서는 __프로토콜__ 이라고 불렀습니다.

> 스몰토크(small takl)는 동적 형, 반영을 지원하는 객체 지향 프로그래밍 언어

__즉, 클래스는 여러 프로토콜을 구현해서 객체가 여러 역할을 할 수 있게 만들 수 있습니다.__

프로토콜은 인터페이스지만 비공식적입니다. 문서와 관례에 따라 정의되지만, 공식 인터페이스처럼 강제할 수 없습니다. 또 특정 클래스에서 부분적으로 구현할 수도 있습니다.

## 11.2 파이썬은 시퀀스를 찾아낸다

파이썬 데이터 모델은 __가능한 많은 핵심 프로토콜과 협업하겠다__는 철학을 가지고 있습니다.

![sequence uml](../images/11_1.png)

ABC로 정의된 공식적인 Sequence 인터페이스를 보여줍니다.

In [5]:
## ex 11-3
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [6]:
f = Foo()
f[1]

10

In [7]:
for i in f: print(i)

0
10
20


In [8]:
20 in f, 15 in f

(True, False)

`__iter__()` 메서드가 구현되지 않아도, 대체 수단인 `__getitem__()` 메서드가 구현되어 있으므로 iterable합니다. 

파이썬 인터프리터는 0부터 시작하는 정수 인덱스로 `__getitem__()` 메서드를 호출하여 객체 반복을 시도하기 때문입니다.

또 `__contains__()`메서드가 구현되어 있지 않아도, `in`연산자를 사용할 수 있습니다.

In [9]:
## 11-4
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks
                                        for suit in self.suits]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

파이썬은 약간이라도 시퀀스를 닮은 객체는 모두 특별하게 처리합니다. 

객체를 반복하기 위해 파이썬 인터프리터는 두 가지 다른 메서드를 시도하므로, 반복은 덕 타이핑의 극단적인 예를 보여줍니다. 

In [10]:
fd = FrenchDeck()

for _ in fd[:5]:
    print(_)

Card(rank='2', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='2', suit='hearts')
Card(rank='3', suit='spades')


## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

규정된 프로토콜을 잘 따르면, 덕 타이핑 덕에 기존 표준 라이브러리와 서드파티 코드(오픈 소스 코드 같은..) 를 활용할 수 있는 가능성이 높아집니다. 

In [11]:
from random import shuffle

shuffle(fd)

TypeError: 'FrenchDeck' object does not support item assignment

위의 `FrenchDeck` 클래스는 불변 시퀀스 프로토콜만 구현하기 때문입니다. 가변 시퀀스는 `__setitem__()` 메서드도 지원해야 합니다.

파이썬은 동적 언어이므로 코드를 대화형 콘솔에서 실행하는 동안에도 이 문제를 수정할 수 있습니다. 

In [12]:
# ex 11-6 멍키 패칭
def set_card(deck, position, card):
    deck._cards[position] = card
    

FrenchDeck.__setitem__ = set_card

shuffle(fd)

In [13]:
fd[:5]

[Card(rank='2', suit='hearts'),
 Card(rank='A', suit='diamonds'),
 Card(rank='9', suit='diamonds'),
 Card(rank='10', suit='hearts'),
 Card(rank='5', suit='diamonds')]

`__setitem__` 시그너처

```
object.__setitem__(self, key, value)

self[key] 로의 대입을 구현하기 위해 호출됩니다. __getitem__() 과 같은 주의가 필요합니다. 매핑의 경우에는, 객체가 키에 대해 값의 변경이나 새 키의 추가를 허락할 경우, 시퀀스의 경우는 항목이 교체될 수 있을 때만 구현되어야 합니다. 잘못된 key 값의 경우는 __getitem__() 에서와 같은 예외를 일으켜야 합니다.
```

이렇게 소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위를 __멍키 패칭__ 이라 합니다.

이렇게 `random.shuffle()` 함수는 자신이 받는 인수의 자료형에 대해 신경 쓰지 않습니다. 단지 받은 객체가 일부 가변 시퀀스 프로토콜을 구현하고 있으면 될 뿐입니다. 

심지어 소스코드에 없고 나중에 프로토콜을 얻어도 가능하게 합니다.

## 11.4 알렉스 마르텔리의 물새

### 물새와 ABC

1. __덕타이핑__ 은 객체의 실제 자료형보단, 그 객체의 인터페이스에 초점을 둡니다. 
        ex. 조엘은 건축공학을 전공했지만, 코딩을 하기에 개발자 꿈나무이다

2. 하지만 때로는 근본을 따질 일이 있습니다. 따라서 __구스 타이핑__ 으로 이를 보완하고자 합니다.
        ex. 조엘은 개발자 꿈나무가 아니다 ㅜㅜ
        
3. __구스 타이핑__ 은 cls가 추상 베이스 클래스인 경우, 즉 cls의 메타클래스가 `abc.ABCMeta`인 경우에는 `isinstance(obj, cls)`를 써도 좋다는 것을 의미합니다.


4. ABC의 서브클래스로 인식시키기 위해 `register()`라는 클래스 메서드를 이용할 수 있지만, 이는 많은 주의를 필요로합니다.


5. 등록하지 않고도 서브클래스로 인식시킬 수 있습니다. 

In [14]:
class Strugle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Strugle(), abc.Sized)

True

> `abc.Sized` 클래스는 `Struggle`을 일종의 서브클래스로 인식합니다. 단지 `__len__()`만 구현하면 되며, 등록할 필요도 없습니다.

6. ABC가 표현하는 개념을 실현하는 클래스(`numbers`, `collections.abc`)를 구현할 때는 해당 ABC를 상속하거나 해당 ABC에 등록하길 바랍니다. 


7. 배포용 코드에서 절대로 ABC나 메타클래스를 직접 구현하지 말아야 합니다. 혹여나 이런 생각이 든다면 __멋진 망치를 산 사람이 모든게 못으로 보이는 현상__ 입니다.

알렉스는 필요한 메서드를 구현하는 것보다 __ABC를 상속하는 것__ 이 낫다고 강조합니다. 또 상속은 __개발자의 의도__ 를 명확히 나타냅니다. 가상 서브 클래스의 등록을 통해서도 의도가 명확히 드러날 수 있습니다.

ABC의 서브클래스인지 테스트하기 위해 `isinstance()`와 `issubclass()` 함수를 사용하는 것도 개선되었습니다. 

과거에는 이 함수들이 덕 타이핑에 해를 끼쳤지만, ABC가 등장하면서 융통성이 향상되었습니다.

ABC를 사용하더라도 `isinstance()`를 너무 많이 사용하는 것은 구립니다. 일반적으로 `if/elif/elif` 블록에서 계속 `isinstance()`를 검사하는 것은 좋지 않습니다. 그런 경우 __다형성__ 을 사용해야 합니다. 즉 인터프리터가 적절한 메서드를 호출할 수 있게 구현해야 합니다.

In [15]:
## ex 11-7 문자열이나 문자열의 반복 가능형을 처리하기 위한 덕 타이핑
field_names = 'joel jinny jason luna'
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass

field_names = tuple(field_names)

print(field_names)

('joel', 'jinny', 'jason', 'luna')


## 11.5 ABC 상속하기

In [16]:
## ex 11-8
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamons clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

파이썬은 모듈을 로딩하거나 컴파일 할 때가 아니라, 실행 도중 실제로 `FrenchDeck2`객체를 생성할 대 추상 메서드의 구현 여부를 확인합니다. 이때 추상메서드 중 하나라도 구현되어 있지 않으면 `TypeError`를 발생시킵니다. 

따라서 우리는 `__delitem__()`과 `insert()` 메서드를 구현해야 했습니다. `NutableSequence`ABC가 요구하는 사항이기 때문입니다.

In [17]:
isinstance(FrenchDeck2(), collections.abc.MutableSequence)

True

![11_2](../images/11_2.png)

조금 기울어져 있는 글씨체 친구들이 추상 메서드 입니다. ABC를 상속받기 위해서는 이런 애들을 구현해줘야 상속받을 수 있습니다.

## 11.6 표준 라이브러리의 ABC

### 11.6.1 `collections.abc`의 ABC

![img](../images/11_3.png)

파이썬 3.4 버전의 `collections.abc`에 정의된 16개의 ABC를 UML 클래스 다이어그램으로 보여줍니다. 

__Iterable, Container, Sized__

- 모든 컬렉션은 이 ABC를 상속하거나, 적어도 호환되는 프로토콜을 구현해야 합니다. 
- Iterable : `__iter__()`
- Container : `__contains__()`
- Sized : `__len__()`

__Sequence, Mapping, Set__

- 주요 불변 컬렉션형 
- 각기 가변형 서브클래스 존재

__MappingView__

- `items()`메서드 에서 반환된 객체 : ItemView 상속
- `keys()`메서드 에서 반환된 객체 : KeyView 상속
- `valuse()`메서드 에서 반환된 객체 : ValueView 상속

__Callable, Hashable__

- 객체가 호출가능하거나 해시할 수 있는지 판단하기 위해 `isinstance()`와 같이 쓰임

__Iterator__

- Iterable 상속, 14장에서 설명

### 11.6.2 ABC의 숫자탑

numbers 패키지는 숫자 탑을 정의합니다. 

- Number
- Complex
- Real
- Rational
- Integral

`Number`가 최상위 슈퍼클래스이며, 아래로 내려갑니다.

In [18]:
import numbers

isinstance(1, numbers.Integral)

True

In [19]:
isinstance(True, numbers.Integral)

True

In [20]:
isinstance(2., numbers.Integral)

False

In [21]:
isinstance(1, numbers.Real)

True

In [22]:
isinstance(True, numbers.Real)

True

In [23]:
isinstance(.25, numbers.Real)

True